# The py_aff3ct 'hello world'

This tutorial presents the steps for playing with `aff3ct`. 

Before going farther, we should import the `py_aff3ct`library. The sys library is requiered to add the library path to your path. For this example, we will also need the `numpy` library.

In [1]:
import sys  
sys.path.insert(0, '../build')

import py_aff3ct
py_aff3ct.disable_color()

import numpy as np
import math

The `py_aff3ct` library is composed of `modules`. Globally, a module is a device that can process data. A data processing is called a `task` in the `aff3ct` language. Tasks exchange data through `sockets`.

Let start by building a source for generating random binary sequences of length 8 and displaying its `tasks` and `sockets`.

In [2]:
src = py_aff3ct.module.source.Source_random(8)

Now we can generate the bits randomly by executing the task named `generate`.

In [3]:
src('generate').info()
src('generate').exec()

- Task 
	- Name         : generate
	- Sockets      : 
	- Socket 0
		- Name              : U_K
		- Type              : out
		- Elements per frame: 8
		- Data type         : int32
	- Socket 1
		- Name              : status
		- Type              : out
		- Elements per frame: 1
		- Data type         : int32



The generated bits are contained in the socket `U_K` and can be accessed from your python code as a NumPy array using the `module['task_name::socket_name']` syntax. 

In [11]:
bits = src['generate::U_K'][:]
print("bits = ", bits)

bits =  [[0 0 0 1 0 1 1 0]]


We want now to modulate the bits of `U_K` using BPSK modulation. So, we start by building a BPSK `modem` (which is a also a `module`).

In [5]:
mod = py_aff3ct.module.modem.Modem_BPSK(8)

When asking for more details about the task `modulate` of object `mod` we can see that this task has one input socket called `X_N1`.

In [6]:
mod('modulate').info()

- Task 
	- Name         : modulate
	- Sockets      : 
	- Socket 0
		- Name              : X_N1
		- Type              : in
		- Elements per frame: 8
		- Data type         : int32
	- Socket 1
		- Name              : X_N2
		- Type              : out
		- Elements per frame: 8
		- Data type         : float32
	- Socket 2
		- Name              : status
		- Type              : out
		- Elements per frame: 1
		- Data type         : int32



To link the source `src` and the modulator `mod`, the output socket `U_K` of the task `generate` of the module `src` should be "binded" to the input socket `X_N1` of the task `modulate` of the module `mod`.

In [7]:
 mod['modulate::X_N1'].bind(src['generate::U_K'])

Now we can execute the processing sequence.

In [8]:
src('generate').exec()
mod('modulate').exec()
bits    = src['generate::U_K'][:]
symbols = mod['modulate::X_N2'][:]
print("bits    = ", bits)
print("symbols = ", symbols)

bits    =  [[1 1 1 0 0 1 0 0]]
symbols =  [[-1. -1. -1.  1.  1. -1.  1.  1.]]


You can see that the data is automatically transfered from the output of `generate` to the input of `modulate`. The data printing can be handled automatically by setting the `debug` property of the tasks to `true`.

In [9]:
src('generate').debug = True
mod('modulate').debug = True
src('generate').exec()
mod('modulate').exec()

# Source_random::generate(int32 U_K[8])
# {OUT} U_K = [    0,     0,     0,     1,     0,     1,     1,     0]
# Returned status: [0 'SUCCESS']
#
# Modem_BPSK::modulate(const int32 X_N1[8], float32 X_N2[8])
# {IN}  X_N1 = [    0,     0,     0,     1,     0,     1,     1,     0]
# {OUT} X_N2 = [ 1.00,  1.00,  1.00, -1.00,  1.00, -1.00, -1.00,  1.00]
# Returned status: [0 'SUCCESS']
#
